Новости были взяты с сайта https://www.livejournal.com/media

В качестве ключевых слов использовались теги под статьёй

In [1]:
# импорты
import pandas as pd
import RAKE
from summa import keywords
from keybert import KeyBERT
import nltk
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from nltk.corpus import stopwords

stop = stopwords.words('russian')

Функции

In [2]:
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [3]:
def repair_tag(tag_list):
    for tag in range(len(tag_list)):
        if tag_list[tag] in ['ADJS', 'ADJF']:
            tag_list[tag] = 'ADJ'
        if tag_list[tag] in ['COMP', 'PRED']:
            tag_list[tag] = 'ADVB'
        if tag_list[tag] in ['INFN', 'PRTF', 'PRTS', 'GRND']:
            tag_list[tag] = 'VERB'
            
    return tag_list

In [4]:
def filtrate(kw_list, POS=[]):
    filtered = kw_list
    if POS != []:
        filtered = []
        for l in range(len(kw_list)):
            if repair_tag([str(m.parse(w)[0].tag.POS) for w in kw_list[l].split(' ')]) == POS:
                filtered += [kw_list[l]]
    
    return filtered

In [5]:
def cross_count(pred, standart, POS=[]):
    cross = list(set(filtrate(pred, POS)) & set(filtrate(standart, POS)))
    return cross

In [6]:
def count_metrics(result_kw, compare_kw, POS=[]):
    texts_n = len(result_kw)
    prec = 0
    recall = 0
    f1 = 0
    for i in range(texts_n):
        cross = cross_count(result_kw[i], compare_kw[i], POS)
        print(cross)
        prec += len(cross)/len(result_kw[i])
        recall += len(cross)/len(compare_kw[i])
        if prec+recall == 0:
            pass
        else:
            f1 += 2*prec*recall/(prec+recall)
    prec = prec/texts_n
    recall = recall/texts_n
    f1 = f1/texts_n
    return (prec, recall, f1)

Загружаем корпус, нормализуем тексты, соединяем ключевые слова в один список 

In [7]:
corpus = pd.read_csv("articles_kw.csv", sep=";")

In [8]:
corpus['Text'] = [normalize_text(t) for t in corpus['Text']]
corpus['KeyWords'] = [normalize_text(t).split(' , ') for t in corpus['KeyWords']]
corpus['MyWords'] = [normalize_text(t).split(' , ') for t in corpus['MyWords']]

In [9]:
corpus

,Text,KeyWords,MyWords
0,охота за кадр : какой выпускник мочь считаться...,"[вуз, карьер, образование, работа, университет]","[выпускник, вуз, рейтинг, успешность]"
1,слишком строгий воспитание сказаться на днк бе...,"[воспитание, депрессия, ребёнок, днк, здоровье...","[днк, строгий воспитание, депрессия, метилиров..."
2,опасный возраст : депрессия у девочека-подрост...,"[девочка, депрессия, ребёнок, подросток, психо...","[депрессия, депрессивный состояние, девочка, п..."
3,как темперамент ребёнок влиять на он успех в ш...,"[образование, психология, темперамент, школа]","[темперамент, ребёнок, личность, школа, особен..."
4,"система поддержка , и почему травма так травми...","[поддержка, психологический помощь, психология...","[травма, ураган, человек, поддержка, психотравма]"
5,как меняться мозг мужчина с появление ребёнок ...,"[ребёнок, мозг, отцовство]","[отец, мозг, ребёнок, родительский поведение, ..."


In [10]:
kw_compare = []
for i in range(len(corpus)):
    kw_compare += [list(set(corpus['KeyWords'][i] + corpus['MyWords'][i]))]

Считаем через RAKE

In [11]:
rake = RAKE.Rake(stop)

In [12]:
kw_rake = []
for i in range(len(corpus)):
    kw_rake += [list(set([w[0] for w in rake.run(corpus['Text'][i], maxWords=2, minFrequency=2)]))]

In [13]:
kw_rake[0]

['например',
 'мочь',
 'рейтинг',
 'карьер',
 'выпускник',
 'кстати',
 'успешность выпускник',
 'хороший',
 'кома']

In [14]:
count_metrics(kw_rake, kw_compare)

['выпускник', 'рейтинг', 'карьер']
[]
['подросток', 'родитель', 'депрессия', 'ребёнок']
['ребёнок', 'школа']
['человек', 'травма', 'ураган']
['мозг', 'мужчина']


(0.17036490980799424, 0.30224867724867727, 0.7514663861329761)

In [15]:
count_metrics(kw_rake, kw_compare, POS=['NOUN'])

['выпускник', 'рейтинг', 'карьер']
[]
['подросток', 'родитель', 'депрессия', 'ребёнок']
['ребёнок', 'школа']
['человек', 'травма', 'ураган']
['мозг', 'мужчина']


(0.17036490980799424, 0.30224867724867727, 0.7514663861329761)

In [16]:
count_metrics(kw_rake, kw_compare, POS=['ADJ','NOUN'])

[]
[]
[]
[]
[]
[]


(0.0, 0.0, 0.0)

In [17]:
list(set(kw_rake[0]) & set(kw_compare[0]))

['выпускник', 'рейтинг', 'карьер']

In [18]:
kw_text_rank = []
for i in range(len(corpus)):
    kw_text_rank += [list(set([w[0] for w in keywords.keywords(normalize_text(corpus['Text'][i]), language='russian', additional_stopwords=stop, scores=True)]))]

In [19]:
kw_text_rank[0]

['влиять',
 'право',
 'специалист',
 'человек',
 'оксфорд достаточно',
 'рыба',
 'отрасль',
 'особенно самый престижный',
 'свой',
 'наплевательск',
 'правило',
 'рейтинг',
 'насколько',
 'работодатель',
 'студент причина',
 'престижность',
 'весь',
 'доверять практически',
 'субъективный',
 'мор',
 'каждый',
 'mater',
 'это',
 'выдающийся',
 'субъективность',
 'сложиться ваш',
 'несколько ключевой',
 'например',
 'предпочтение',
 'специальность',
 'абитуриент',
 'относиться',
 'совершенно',
 'образование',
 'отдельный',
 'диплом',
 'выпускник мочь',
 'вековой',
 'карьер',
 'политехнический',
 'кстати',
 'процесс',
 'хороший российский вуз']

In [20]:
count_metrics(kw_text_rank, kw_compare)

['образование', 'рейтинг', 'карьер']
['родитель', 'метилирование', 'ребёнок', 'днк', 'наказание', 'строгий воспитание']
['ребёнок', 'депрессия', 'девочка']
['личность', 'школа', 'особенность']
['человек', 'травма', 'психотравма', 'поддержка', 'ураган']
['отец', 'ребёнок']


(0.09902786175262185, 0.4492063492063492, 0.5984593193024678)

In [21]:
count_metrics(kw_text_rank, kw_compare, POS=['NOUN'])

['образование', 'рейтинг', 'карьер']
['родитель', 'метилирование', 'ребёнок', 'днк', 'наказание']
['ребёнок', 'депрессия', 'девочка']
['личность', 'школа', 'особенность']
['человек', 'травма', 'психотравма', 'поддержка', 'ураган']
['отец', 'ребёнок']


(0.09261760534236545, 0.43253968253968256, 0.5504447854746926)

In [22]:
count_metrics(kw_text_rank, kw_compare, POS=['ADJ','NOUN'])

[]
['строгий воспитание']
[]
[]
[]
[]


(0.006410256410256411, 0.016666666666666666, 0.0462962962962963)

In [23]:
kw_model = KeyBERT('clips/mfaq')

c:\users\polina\appdata\local\programs\python\python38\lib\site-packages\transformers\configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [24]:
kw_bert_1 = []
for i in range(len(corpus)):
    kw_bert_1 += [list(set([w[0] for w in kw_model.extract_keywords(corpus['Text'][i])]))]

In [25]:
kw_bert_1[0]

['топ', 'престиж', 'рейтинг', 'престижность', 'заслуживать']

In [26]:
count_metrics(kw_bert_1, kw_compare)

['рейтинг']
[]
['депрессия', 'подросток']
['психология']
['поддержка']
['отец', 'мозг']


(0.2333333333333333, 0.15806878306878305, 0.5413076960194714)

In [27]:
count_metrics(kw_bert_1, kw_compare, POS=['NOUN'])

['рейтинг']
[]
['депрессия', 'подросток']
['психология']
['поддержка']
['отец', 'мозг']


(0.2333333333333333, 0.15806878306878305, 0.5413076960194714)

In [28]:
kw_bert_2 = []
for i in range(len(corpus)):
    kw_bert_2 += [list(set([w[0] for w in kw_model.extract_keywords(normalize_text(corpus['Text'][i]), keyphrase_ngram_range=(1, 2), stop_words=stop)]))]

In [29]:
count_metrics(kw_bert_2, kw_compare, POS=['ADJ', 'NOUN'])

[]
['строгий воспитание']
[]
[]
[]
[]


(0.03333333333333333, 0.016666666666666666, 0.1111111111111111)

В конце концов оказалось достаточно сложно оценить качество выделения слов, потому что составленный стандарт далёк* от идеала... Однако достаточно очевидно, что на маленьких текстах с одной стороны "значимых" слов слишком много т.к. даже одно вхождение уже значимо, при этом слово действительно может чаще остальных встречаться и при этом быть не тематическим. Мне кажется можно попробовать заранее задавать большую тему одним-двумя словами и по какой-нибудь можели обученой на большом корпусе текстов с разной тематикой смотреть на сколько ключевое слово вообще связано с общей тематикой текста (с точки зрения их семантики вообще)

*исходные теги в источнике были достаточно плохи, да и в ручную я совершила ошибку, не включив те же одиночные прилагательные, так что метрики здесь не могут показать значимого результата